In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

import PIL
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import os
import random
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf

from numpy import save
from numpy import load

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow import keras
from tensorflow.keras import Input, Model

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten

In [2]:
X = load('G:/Desktop/monkey dataset/data/data224(3).npy')
y = load('G:/Desktop/monkey dataset/data/y224(3)(all).npy')

In [3]:
x_test = load('G:/Desktop/monkey dataset/data/test_data224(3).npy')
y_test = load('G:/Desktop/monkey dataset/data/test_y224(3)(all).npy')

In [4]:
X.shape

(1098, 224, 224, 3)

In [5]:
y.shape

(1098,)

In [6]:
x_train = X[:950]
y_train = y[:950]
x_dev = X[950:]
y_dev = y[950:]

In [7]:
x_test.shape

(272, 224, 224, 3)

In [8]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
dev_dataset = tf.data.Dataset.from_tensor_slices((x_dev, y_dev))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

In [9]:
BATCH_SIZE = 4
SHUFFLE_BUFFER_SIZE = 1000

train_batches = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = dev_dataset.batch(BATCH_SIZE)
test_batches = test_dataset.batch(BATCH_SIZE)

In [10]:
IMG_SHAPE = (224, 224, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.VGG19(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

80142336/80134624 [==============================] - 138s 2us/step


In [11]:
x = base_model.get_layer('block4_pool').output
prediction = tf.keras.layers.Flatten()(x)

In [12]:
model = Model(inputs=base_model.input, outputs=prediction)

In [13]:
%%time
block4_pool_features = model.predict(x_train)
block4_pool_features_dev = model.predict(x_dev)
block4_pool_features_test = model.predict(x_test)

Wall time: 34.1 s


In [14]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

In [15]:
block4_pool_features.shape

(950, 100352)

In [16]:
block4_pool_features_test.shape

(272, 100352)

In [17]:
%%time
svm = LinearSVC()
svm.fit(block4_pool_features, y_train)

Wall time: 3min 37s


C:\Users\Dindar\anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [18]:
preds = svm.predict(block4_pool_features_dev)
preds_test = svm.predict(block4_pool_features_test)

In [19]:
accuracy_score(y_dev, preds), accuracy_score(y_test, preds_test)

(0.722972972972973, 0.7242647058823529)